In [ ]:

! pip install dgl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
"""
  DGL使用一个唯一的整数来表示一个节点，称为点ID；并用对应的两个端点ID表示一条边。同时，DGL也会根据边被添加的顺序，
  给每条边分配一个唯一的整数编号，称为边ID。节点和边的ID都是从0开始构建的。在DGL的图里，所有的边都是有方向的 
"""
import dgl
import torch as th

# 边 0-1 0-2 0-3 1-3
u,v = th.tensor([0,0,0,1]),th.tensor([1,2,3,3])
g = dgl.graph((u,v))
print("the graph of g",g)

# 获取节点的id
print(g.nodes())

# 获取边对应的端点
print(g.edges())

# 获取边对应端点和边ID
print(g.edges(form='all'))

# 如果具有最大id的节点没有边，在创建图的时候，用户需要明确地指明节点数量
g = dgl.graph((u,v),num_nodes=8)

the graph of g Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})
tensor([0, 1, 2, 3])
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))


In [ ]:
print(g)

Graph(num_nodes=8, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
# 对于无向图，需要为每条边都创建两个方向的边
bg = dgl.to_bidirected(g)
bg.edges()

(tensor([0, 0, 0, 1, 1, 2, 3, 3]), tensor([1, 2, 3, 0, 3, 0, 0, 1]))

In [ ]:
# DGLGraph 对象的节点和边可具有多个用户定义的、可命名的特征，以存储图的节点和边的属性
# 可通过ndata，edata访问这些特征。

g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0]))
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
th.ones(3,3)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
th.ones(6,3)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
g.ndata['x'] = th.ones(g.num_nodes(),3)
g.edata['x'] = th.ones(g.num_edges(),dtype=th.int32)
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})

In [ ]:
# 不同名称的特征可以有不同形状
g.ndata['y'] = th.randn(g.num_nodes(),5)
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})

In [ ]:
# 获取节点x的特征
g.ndata['x'][1]
g.ndata['y'][1]

tensor([-1.2478, -0.3388, -1.3215,  0.2925, -2.4185])

In [ ]:
# 获取边0 3 的特征
g.edata['x'][th.tensor([0,3])]

tensor([1, 1], dtype=torch.int32)

In [ ]:
# 对于加权图，可以将权重存储为一个边特征
edges = th.tensor([0,0,0,1]),th.tensor([1,2,3,3])
weights = th.tensor([0.1,0.6,0.9,0.7])
g = dgl.graph(edges)
g.edata['W'] = weights
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={'W': Scheme(shape=(), dtype=torch.float32)})

In [ ]:
g.edata['W']

tensor([0.1000, 0.6000, 0.9000, 0.7000])

In [ ]:
# 从外部源创建图
"""
  可以从外部来源构造一个 DGLGraph 对象，包括
  从用于图和稀疏矩阵的外部Python库（NetworkX 和 SciPy）创建而来。
  从磁盘加载图数据。
"""
import scipy.sparse as sp
# 百分之五非零项
spmat =sp.rand(100,100,density=0.05)
dgl.from_scipy(spmat)

Graph(num_nodes=100, num_edges=500,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
import networkx as nx

# 0 1 2 3 4 
nx_g = nx.path_graph(5)
dgl.from_networkx(nx_g)

Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
# 磁盘加载图
# 逗号分隔值
# https://github.com/dglai/WWW20-Hands-on-Tutorial/blob/master/basic_tasks/1_load_data.ipynb

In [ ]:
"""
  异构图
  异构图里可以有不同类型的节点和边。这些不同类型的节点和边具有独立的ID空间和特征。
  在DGL中，一个异构图由一系列子图构成，一个子图对应一种关系。每个关系由一个字符串三元组
  定义 (源节点类型, 边类型, 目标节点类型) 。由于这里的关系定义消除了边类型的歧义，DGL称它们为规范边类型。
"""
# 创建一个具有三种节点类型和三种边类型的异构图
graph_data ={
    ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
    ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
    ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)

In [ ]:
g.ntypes

['disease', 'drug', 'gene']

In [ ]:
g.etypes


['interacts', 'interacts', 'treats']

In [ ]:
# 构图和二分图只是一种特殊的异构图，它们只包括一种关系。
dgl.heterograph({('node_type', 'edge_type', 'node_type'): (u, v)})
dgl.heterograph({('source_type', 'edge_type', 'destination_type'): (u, v)})

Graph(num_nodes={'destination_type': 4, 'source_type': 2},
      num_edges={('source_type', 'edge_type', 'destination_type'): 4},
      metagraph=[('source_type', 'destination_type', 'edge_type')])

In [ ]:
"""
  与异构图相关联的 metagraph 就是图的模式。它指定节点集和节点之间的边的类型约束。 
  metagraph 中的一个节点 u 对应于相关异构图中的一个节点类型。
  metagraph 中的边 (u,v) 表示在相关异构图中存在从 u 型节点到 v 型节点的边。
"""
g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [ ]:
"""
  当引入多种节点和边类型后，用户在调用DGLGraph API以获取特定类型的信息时，
  需要指定具体的节点和边类型。此外，不同类型的节点和边具有单独的ID。
"""
# 获取图中所有节点数量
g.num_nodes()

10

In [ ]:
# 获取drug节点的数量
g.num_nodes('drug')

3

In [ ]:
# 不同类型的节点有单独的id，因此没有指定节点类型就没有明确返回值
g.nodes()

DGLError: ignored

In [ ]:
g.nodes('drug')

In [ ]:
"""
  为了设置/获取特定节点和边类型的特征，DGL提供了两种新类型的语法： 
  g.nodes[‘node_type’].data[‘feat_name’] 和 g.edges[‘edge_type’].data[‘feat_name’] 。
"""
# 设置/获取 drug 类型的节点的hv特征
g.nodes['drug'].data['hv'] = th.ones(3,1)
g.nodes['drug'].data['hv']

In [ ]:
# 设置/获取"treats"类型的边的"he"特征
g.edges['treats'].data['he'] = th.zeros(1, 1)
g.edges['treats'].data['he']

In [ ]:
# 如果图里只有一种节点或边类型，则不需要指定节点或边的类型。
g = dgl.heterograph({
  ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
  ('drug', 'is similar', 'drug'): (th.tensor([0, 1]), th.tensor([2, 3]))
})
g.nodes()

In [ ]:
# 从磁盘加载异构图
# 一种存储异构图的常见方法是在不同的CSV文件中存储不同类型的节点和边。下面是一个例子。
"""
  # 数据文件夹
data/
|-- drug.csv        # drug节点
|-- gene.csv        # gene节点
|-- disease.csv     # disease节点
|-- drug-interact-drug.csv  # drug-drug相互作用边
|-- drug-interact-gene.csv  # drug-gene相互作用边
|-- drug-treat-disease.csv  # drug-disease治疗边

与同构图的情况类似，用户可以使用像Pandas这样的包先将CSV文件解析为numpy数组或框架张量，再构建一个关系字典，
并用它构造一个异构图。 这种方法也适用于其他流行的文件格式，比如GML或JSON。
"""

In [ ]:
# 边类型子图
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})
g.nodes['drug'].data['hv'] = th.ones(3, 1)

In [ ]:
# 保留关系 ('drug', 'interacts', 'drug') 和 ('drug', 'treats', 'disease') 。
# 'drug' 和 'disease' 类型的节点也会被保留
eg = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                ('drug', 'treats', 'disease')])
eg

In [ ]:
# 相关的特征也会被拷贝
eg.nodes['drug'].data['hv']

In [ ]:
# DGL允许使用 dgl.DGLGraph.to_homogeneous() API将异构图转换为同构图
# 用从0开始的连续整数重新标记所有类型的节点和边。
# 对所有的节点和边合并用户指定的特征。
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))})
g.nodes['drug'].data['hv'] = th.zeros(3, 1)
g.nodes['disease'].data['hv'] = th.ones(3, 1)
g.edges['interacts'].data['he'] = th.zeros(2, 1)
g.edges['treats'].data['he'] = th.zeros(1, 2)

In [ ]:
# 默认情况下不进行特征合并
hg = dgl.to_homogeneous(g)
'hv' in hg.ndata

In [ ]:
# https://docs.dgl.ai/en/1.0.x/guide_cn/graph-heterogeneous.html